In [13]:
import numpy as np
import os
import pickle

task_list = [ 'autoencoder','class_1000', 'class_places','curvature', 'edge2d', 'edge3d',\
                         'keypoint2d', 'keypoint3d','reshade', 'rgb2depth', 'rgb2mist', 'rgb2sfnorm','room_layout' , \
                         'segment25d', 'segment2d', 'segmentsemantic','vanishing_point']
list_of_tasks = 'autoencoder curvature denoise edge2d edge3d \
keypoint2d keypoint3d colorization \
reshade rgb2depth rgb2mist rgb2sfnorm \
room_layout segment25d segment2d vanishing_point \
segmentsemantic class_1000 class_places inpainting_whole pascal_voc_segmentation'
task_list_new = list_of_tasks.split(' ')
indices = []
for task in task_list:
    indices.append(task_list_new.index(task))
indices = np.array(indices)
print(indices)

[ 0 17 18  1  3  4  5  6  8  9 10 11 12 13 14 16 15]


In [14]:
from scipy.stats import spearmanr,pearsonr
path = './../results/DDScomparison_pascal/pascal_5000/'
kernel_affinities =  np.load(path+'kernels.npy',allow_pickle=True).item()
rdm_affinities = np.load(path+'rdms.npy',allow_pickle=True).item()

corr_value = {}
performance = [0.590164306133055,0.649290395668602,0.652942936208706,0.647184483742267,0.600769184861427,\
                           0.649696268320798, 0.5994201374950169,0.647529319500711,\
                           0.6300454561362,0.6487250234464327,0.643817731132111,0.647786678981121,0.625850319733945,\
                           0.656534260134841,0.629964850307006,0.648764305659868, 0.589181267604318];

for kernel,kernel_affinity in kernel_affinities.items():
    corr_value[kernel] = {}
    for f_ablation,f_ablation_affinity in kernel_affinity.items():
        corr_value_matrix = np.zeros(f_ablation_affinity.shape[0])
        for iter in range(f_ablation_affinity.shape[0]):
            temp,_ = spearmanr(performance, f_ablation_affinity[iter,indices])
            corr_value_matrix[iter] = temp
        corr_value[kernel][f_ablation] = corr_value_matrix
            

for rdm,rdm_affinity in rdm_affinities.items():
    corr_value[rdm] = {}
    for f_ablation,f_ablation_affinity in rdm_affinity.items():
        corr_value_matrix = np.zeros(f_ablation_affinity.shape[0])
        for iter in range(f_ablation_affinity.shape[0]):
            temp,_ = spearmanr(performance, f_ablation_affinity[iter,indices])
            corr_value_matrix[iter] = temp
        corr_value[rdm][f_ablation] = corr_value_matrix

In [15]:
max_corr = 0
mean_corr = {}
for dist,corr_dist in corr_value.items():
    mean_corr[dist] = {}
    for f_ablation,corr_f_ablation in corr_dist.items():
        mean_corr[dist][f_ablation] = round(np.mean(corr_f_ablation),3)
        if np.mean(corr_f_ablation) > max_corr:
            max_corr = np.mean(corr_f_ablation)
            max_combo = [dist,f_ablation]

print(max_corr,max_combo)


0.7916666666666667 ['linear', 'znorm']


In [16]:
import pandas as pd
dfObj = pd.DataFrame(mean_corr)
print(dfObj)

         rbf    lap  linear  pearson  euclidean  cosine
znorm  0.752  0.789   0.792    0.743      0.787   0.751
